In [12]:
import os
import re

def build_component_map(components_base_path):
    component_map = {}
    for root, _, files in os.walk(components_base_path):
        for file in files:
            if file.endswith('.yaml'):
                # Create a map entry using the filename without extension as the key
                key = os.path.splitext(file)[0]
                # Store the full path for direct access
                component_map[key] = os.path.join(root, file)
    return component_map

def update_file_references(file_path, component_map, components_base_path):
    with open(file_path, 'r') as file:
        content = file.read()

    updated = False
    # Match all $ref entries
    refs = re.findall(r"\$ref: '([^']+)'", content)
    for ref in refs:
        # Derive the key from the reference to lookup in the component map
        ref_key = os.path.basename(ref).split('.')[0]
        if ref_key in component_map:
            # Get the absolute path of the referenced component
            ref_abs_path = component_map[ref_key]
            # Calculate the relative path from the current file to the referenced component
            rel_path_to_component = os.path.relpath(ref_abs_path, start=os.path.dirname(file_path))
            # Ensure the use of forward slashes for consistency
            rel_path_to_component = rel_path_to_component.replace(os.path.sep, '/')
            # Prefix with './' if not already present
            if not rel_path_to_component.startswith('.'):
                rel_path_to_component = './' + rel_path_to_component
            new_ref = f"$ref: '{rel_path_to_component}'"
            content = content.replace(f"$ref: '{ref}'", new_ref)
            updated = True

    if updated:
        with open(file_path, 'w') as file:
            file.write(content)

def update_project_references(project_root):
    components_base_path = os.path.join(project_root, 'components')
    component_map = build_component_map(components_base_path)

    # Process all YAML files in the project, updating references where necessary
    for root, dirs, files in os.walk(project_root):
        for file in files:
            if file.endswith('.yaml'):
                file_path = os.path.join(root, file)
                update_file_references(file_path, component_map, components_base_path)

project_root = './' # Replace this with your actual project root path
update_project_references(project_root)
